In [ ]:
# 🧠 Bias Buster: AI-Powered News Headline Rewriter & Bias Detector

# STEP 1: Install dependencies
!pip install newspaper3k requests --quiet

# STEP 2: Imports
from newspaper import Article
import requests
import re

# STEP 3: Bias detection logic
def detect_bias(text):
    bias_types = []
    text_lower = text.lower()

    if any(word in text_lower for word in ["clash", "erupts", "tensions rise", "chaos"]):
        bias_types.append("Framing")
    if any(word in text_lower for word in ["officer-involved", "neutralized", "incident"]):
        bias_types.append("Euphemism")
    if re.search(r"\b(shot|killed|struck)\b.*\bby\b", text_lower) is None and " by " not in text_lower:
        bias_types.append("Vague agency / Passive voice")

    return bias_types

def score_bias(bias_types):
    return 50 + 10 * len(bias_types) if bias_types else 0

# STEP 4: Rewrite headline using OpenRouter
def rewrite_with_openrouter(text, api_key):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "openai/gpt-3.5-turbo",  # or mistralai/mistral-7b-instruct
        "messages": [
            {
                "role": "user",
                "content": f"Rewrite this biased headline in objective, direct language:\n\n\"{text}\""
            }
        ],
        "temperature": 0.7
    }

    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=data
        )
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        return "⚠️ Failed to rewrite headline. Please check your API key."

# STEP 5: Extract article title + text
def extract_article(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.title.strip(), article.text.strip()
    except:
        return None, None

# STEP 6: Main UI
print("🧠 Bias Buster Tool")
api_key = input("🔑 Enter your OpenRouter API key: ").strip()

mode = input("Choose mode:\n1 = Analyze a HEADLINE\n2 = Analyze a URL/article\nYour choice: ").strip()

# --- HEADLINE MODE ---
if mode == "1":
    headline = input("📰 Enter headline: ").strip()
    bias_tags = detect_bias(headline)
    score = score_bias(bias_tags)
    rewritten = rewrite_with_openrouter(headline, api_key)

    print("\n🔍 Headline Bias Analysis")
    print(f"📰 Original Headline: {headline}")
    print(f"⚠️ Bias Types Detected: {', '.join(bias_tags) if bias_tags else 'None'}")
    print(f"📊 Estimated Bias Score: {score}/100")
    print(f"📝 Rewritten Headline: {rewritten}")

# --- URL MODE ---
elif mode == "2":
    url = input("🔗 Enter article URL: ").strip()
    title, text = extract_article(url)

    if title is None:
        print("❌ Could not extract article.")
    else:
        combined = title + " " + text
        bias_tags = detect_bias(combined)
        score = score_bias(bias_tags)

        print("\n📰 Article Bias Analysis")
        print(f"📌 Title: {title}")
        print(f"⚠️ Bias Types Detected: {', '.join(bias_tags) if bias_tags else 'None'}")
        print(f"📊 Estimated Bias Score: {score}/100")

        if "framing" in [b.lower() for b in bias_tags] or "euphemism" in [b.lower() for b in bias_tags]:
            print("\n⚠️ Biased phrases may be present in full article.")
        else:
            print("\n✅ Article appears mostly neutral.")
else:
    print("❌ Invalid choice. Please restart and enter 1 or 2.")
